In [55]:
import pandas as pd
pd.set_option('display.max_columns', None)

import requests as rq
import json
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Keys

In [2]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [3]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

### API status

In [4]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [5]:
def get_response(endpoint, headers, params, URL):
    
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    
    # if response.status_code == 200:
    #     data = response.json()
    #     return data
    # else:
    #     print(f"Failed to fetch data, check status code {response.status_code}")

    data = response.json()
    return data

In [6]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [7]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

### Create URLs

In [8]:
def get_url(url_type,
            network = "",
            dex = "",
            pool_address = "",
            token_address = ""):

    url_dict = {
        "new_pools": f"/onchain/networks/new_pools",
        "trending_pools": f"/onchain/networks/{network}/pools",
        "top_pools": f"/onchain/networks/{network}/pools",
        "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools",
        "specific_pool_dex": f"/onchain/networks/{network}/pools/{pool_address}",
        "top_pools_add": f"/onchain/networks/{network}/tokens/{token_address}/pools",
        "token_data": f"/onchain/networks/{network}/tokens/{token_address}",
        "token_info": f"/onchain/networks/{network}/tokens/{token_address}/info"
    }

    return url_dict[url_type]

### Get list of networks

In [9]:
def get_all_networks():    

    networks_list_response = get_response("/onchain/networks",
                                          use_pro, 
                                          "",
                                          PRO_URL)

    return pd.DataFrame(networks_list_response["data"])

In [10]:
df_all_networks = get_all_networks()

In [11]:
df_all_networks[df_all_networks["id"].str.contains("bsc",
                                                   case=False,
                                                   na=False)]

,id,type,attributes
1,bsc,network,"{'name': 'BNB Chain', 'coingecko_asset_platfor..."


### Get top pools for a specific network

In [12]:
def collect_response(list_response):    

    response_all = []

    for response in list_response["data"]:
        all_attributes = response["attributes"]
        daily_tx = all_attributes["transactions"]["h24"]
        rel = response["relationships"]
        base_token_add = rel["base_token"]["data"]["id"]
        
        temp_dict = dict(
            pair = all_attributes["name"],
            dex = rel["dex"]["data"]["id"],
            token_add = base_token_add.split("_")[1],
            pool_add = all_attributes["address"],
            fdv_usd = all_attributes["fdv_usd"],
            market_cap_usd = all_attributes["market_cap_usd"],
            daily_volume = all_attributes["volume_usd"]["h24"],
            daily_price_change = all_attributes["price_change_percentage"]["h24"],
            daily_buys = daily_tx["buys"],
            daily_sells = daily_tx["sells"],
            daily_buyers = daily_tx["buyers"],
            daily_sellers = daily_tx["sellers"]             
        )
        
        response_all.append(temp_dict)

    return response_all

In [13]:
def get_top_pools_network(network, sort_by_col):

    target_url = get_url("top_pools", network)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [14]:
get_top_pools_network("solana", "market_cap_usd").head(5)

,pair,dex,token_add,pool_add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
1,pippin / SOL,raydium,Dfh5DzRgSvvCFDoYc2ciTkMrbDfRKybA4SoFbPmApump,8WwcNqdZjCY5Pt7AkhupAFknV2txca9sq6YBkGzLbvdt,62137217.4408412,62137217.5077773,43120781.0288302,122.647,51548,66723,1443,1924
7,USDT / SOL,pancakeswap-v3-solana,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,22HUWiJaTNph96KQTKZVy2wg8KzfCems5nyW7E5H5J6w,2740515742.98512,184440704324.128,10229433.0526827,0.056,49436,36936,4358,2549
0,SOL / USDC,pancakeswap-v3-solana,So11111111111111111111111111111111111111112,DJNtGuBGEQiUCWE8F981M2C3ZghZt2XLD8f2sQdZ6rsZ,1762011982.23511,1762011982.23511,35774377.5274567,3.882,61118,61260,1848,2299
2,SOL / USDT 0.01%,raydium-clmm,So11111111111111111111111111111111111111112,3nMFwZXwY1s1M5s8vYAHqd4wGs4iSxXE4LRoUMMYqEgF,1762011982.23511,1762011982.23511,16128294.2761907,3.882,52426,60377,2944,3241
4,SOL / USDC 0.01%,raydium-clmm,So11111111111111111111111111111111111111112,8sLbNZoA1cfnvMJLPfp98ZLAnFSYCFApfJKMbiXNLwxj,1762011982.23511,1762011982.23511,4498589.57966194,3.882,50637,54500,2301,2540


### Get new pools

In [15]:
def get_new_pools(sort_by_col):

    target_url = get_url("new_pools")

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [16]:
get_new_pools("market_cap_usd").head(10)

,pair,dex,token_add,pool_add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
0,Opus 4.5 / SOL,pump-fun,6j3BBRskuPskqjF2RvQnb7y7yvev9vPscz7v7Bqwpump,GRWScjABh1JDGs1fEeTZYoh5gPMoqFW8dr31fnnGNGm6,None,None,2.7481391081,0,1,1,1,1
1,GDOG / SOL,pump-fun,FoHsBDPPEVsALkpu7vpvyGPhEwhkbAyut3zGUyEipump,HWf434dx8zBDVfPNbVJ6poQGXMYsMDBSuBagDFzDWP7x,None,None,976.7760405836,-20.408,3,5,3,5
2,GDOG / SOL,meteora-damm-v2,rw5y1uyimDZL8GnbFb7ppPAkXuy9Ek9wAGrbQiEVq2C,7VjTcozYVxQncTaLh4YPuuuNCk7ogV1hk5yKtip4Yiwy,47795.93963,None,183.4515744912,2.326,9,9,9,9
3,Opus 4.5 / SOL,pump-fun,Fj3p5ncSsSnPeLrSdi4Fu5yyJ8QGiMW78MwWH2VMjiNu,3cKW56WZdrcxkyGhj2aX2U5F3KhjkDSGBkgwCd6Ge4EJ,4484.437185,None,1374.4331345823,0.053,1,1,1,1
4,WORKSUCKS / USDC,meteora-dbc,2tZyzRHnV4YzNhxgPBEeV4B8q2D1Ms1LqNrkMswcmoon,8hFAGQ4HSNpDAVyJrgykRAK7vNcRf3txPhLXqWkAdRX8,5201.4646615137,None,99.9412823037,2.029,2,0,1,0
5,MONAD / SOL,meteora-damm-v2,DKtdqVLLLNdhjNnDtcXtGF2LXRop6D3Qpkmq9vApCSya,83FyyGU1rGDYvKAsac7d1fy52qjeeCYYJxHNfrdvvn4b,20470.63823,None,0.2059043396,153.716,2,0,1,0
6,coin / SOL,pump-fun,9s6zGLusfFP62mzabTbyp6PgiWFLXxSa8bEmiARmpump,4kS5qMxVtoAgfrM1UE2CyoSfPBas1BvmutEQFzhv8tD5,3905.5183804294,None,62.3260102367,1.326,3,1,3,1
7,JSEK / WETH,uniswap-v2-base,0xabb9e1ebf573021e1b7cd24690dc90f40634d3e0,0x2f4f72f4f2d874f2c5c6263f05ab323db0202b00,44.32993666,None,44.3255036673,0,0,1,0,1
8,GDOG / SOL,pump-fun,D1Rfaoic33bTLzBZAF1TnSsx2Z53ELoE7b2R8b66X9Q1,Bt8HHMT6QDvqigJgCubBEUkcvQRjf5d2hVHyw72ZsTVF,4020.7509390149,None,1217.1504650071,-27.125,1,2,1,2
9,SHIT / SOL,pump-fun,3JA6sU8MsJs74oVxgyNKD8xo9S4DWrk4GAZTocBrpump,9iKG5x4ZGwik7vWA5Vswcc3iUrWFTAnqEJ3m4y3YRN3e,3969.4228208677,None,86.4166942678,1.655,4,3,3,2


### Get data for a specific pool address

In [17]:
def collect_pool_response(list_response):    

    response = list_response["data"]
    all_attributes = response["attributes"]
    daily_tx = all_attributes["transactions"]["h24"]
    rel = response["relationships"]
        
    response_dict = dict(
        pair = all_attributes["name"],
        dex = rel["dex"]["data"]["id"],
        add = all_attributes["address"],
        fdv_usd = all_attributes["fdv_usd"],
        market_cap_usd = all_attributes["market_cap_usd"],
        daily_volume = all_attributes["volume_usd"]["h24"],
        daily_price_change = all_attributes["price_change_percentage"]["h24"],
        daily_buys = daily_tx["buys"],
        daily_sells = daily_tx["sells"],
        daily_buyers = daily_tx["buyers"],
        daily_sellers = daily_tx["sellers"]
    )

    return response_dict

In [18]:
def get_pool_data(network, dex, pool_address):

    target_url = get_url("specific_pool_dex", network, dex, pool_address)

    pool_list_response = get_response(target_url,
                                      use_pro, 
                                      "",
                                      PRO_URL)

    pool_all = collect_pool_response(pool_list_response)   

    return pool_all

In [19]:
get_pool_data("solana", "", "FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz")

{'pair': 'PENGU / SOL',
 'dex': 'orca',
 'add': 'FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz',
 'fdv_usd': '839529067.794588',
 'market_cap_usd': '687837996.68532',
 'daily_volume': '626641.529799561',
 'daily_price_change': '4.9',
 'daily_buys': 704,
 'daily_sells': 843,
 'daily_buyers': 161,
 'daily_sellers': 172}

### Get data for a specific token address

In [20]:
def collect_token_response(list_response):    

    response = list_response["data"]
    all_attributes = response["attributes"]
    launchpad_details = all_attributes["launchpad_details"]
        
    response_dict = dict(
       grad_pert = launchpad_details["graduation_percentage"],
       completed = launchpad_details["completed"],
       completed_at = launchpad_details["completed_at"],
       dest_pool = launchpad_details["migrated_destination_pool_address"]
    )

    return response_dict

In [21]:
def get_token_data(network, token_address):

    target_url = get_url("token_data",
                         network,
                         "",
                         "",
                         token_address)

    token_list_response = get_response(target_url,
                                  use_pro, 
                                  "",
                                  PRO_URL)

    token_all = collect_token_response(token_list_response)   

    return token_all

In [42]:
get_token_data("solana", "BdD7EsVWdNbiM5PZJsBEmZj4VdCrC3myCDTHr2G9moon")

{'grad_pert': 7.23,
 'completed': False,
 'completed_at': None,
 'dest_pool': None}

### Track tokens

In [45]:
list_of_tokens = [
    "96nXL5td1DxpySaYK8yvkazfteyi6YwqVJYx4w1tpump",
    "BdD7EsVWdNbiM5PZJsBEmZj4VdCrC3myCDTHr2G9moon",
    "7JPBqCzTyVBWJEL79jBvRYBReCKqBSFU1HQyo3jHwave",
    "DhSx8SRk9A3sNze2B2Pk9dnDsfcgQZufsqyWm4qkwave"       
]

In [63]:
def track_status(list_of_tokens, network):

    all_token_data = []

    for token in list_of_tokens:
        token_data = get_token_data(network, token)
        token_data["token"] = token
        all_token_data.append(token_data)

    df_token = pd.DataFrame(all_token_data)

    # Identify rows with graduation % above a certain threshold
    df_token["above_25"] = df_token["grad_pert"] > 25

    # Move token column to the end
    column_to_move = 'token'

    # Reorder columns
    df_token = (
        df_token[[col for col in df_token.columns
                  if col != column_to_move]
                          + [column_to_move]]
    )

    return df_token 

In [64]:
df_status = track_status(list_of_tokens, "solana")

In [65]:
df_status

,grad_pert,completed,completed_at,dest_pool,above_25,token
0,1.75,False,None,None,False,96nXL5td1DxpySaYK8yvkazfteyi6YwqVJYx4w1tpump
1,7.23,False,None,None,False,BdD7EsVWdNbiM5PZJsBEmZj4VdCrC3myCDTHr2G9moon
2,0.07,False,None,None,False,7JPBqCzTyVBWJEL79jBvRYBReCKqBSFU1HQyo3jHwave
3,27.91,False,None,None,True,DhSx8SRk9A3sNze2B2Pk9dnDsfcgQZufsqyWm4qkwave
